In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/BAPEMO_6346.csv",index_col=0)
data.head()

,cleaned,Emotions
0,history paisley find dark fossil paisley one a...,Fear
1,pastoral wall dense ivy execution zbigniew her...,Fear
2,shaving mirror hand hack skin belong child use...,Fear
3,taxidermist first hand delicate precise know c...,Fear
4,vacate apartment efficient fate eye storm troo...,Fear


In [ ]:
corpus=data['cleaned']
lst_corpus = []
for string in corpus:
    lst_words = string.split()
    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, len(lst_words), 1)]
    lst_corpus.append(lst_grams)

In [ ]:
from tensorflow.keras import models, layers,preprocessing as kprocessing
tokenizer =kprocessing.text.Tokenizer(lower=True,split='',oov_token="NaN",)
tokenizer.fit_on_texts(lst_corpus)
dic_vocabulary = tokenizer.word_index
lst_sequences=tokenizer.texts_to_sequences(lst_corpus)

In [ ]:
X_train_seq =kprocessing.sequence.pad_sequences(lst_sequences,maxlen=100, padding="post", truncating="post")

In [ ]:
y=data['Emotions']
y=y.map({'Fear':0,'Sad':1,'Happy':2,
'Surprise':3,'Angry':4})

In [ ]:

import gensim.downloader
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
import numpy as np
embeddings = np.zeros((len(dic_vocabulary)+1,
50))
for word,idx in dic_vocabulary.items():
    try:
        embeddings[idx]=glove_vectors[word]
    except:
        pass

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X_train_seq, y, test_size=0.2,
random_state=1000)

In [ ]:
y_train_vec=pd.get_dummies(y_train)
y_test_vec=pd.get_dummies(y_test)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Input
input = Input(shape=(100,))
x=layers.Embedding(input_dim=embeddings.shape[0],output_dim=embeddings.shape[1],weights=[embeddings],trainable=False)(input)

In [ ]:
from tensorflow.keras.layers import SimpleRNN,Dense
l1=SimpleRNN(50,return_sequences=False,activation='tanh')(x)
output=Dense(5,activation='softmax')(l1)


In [ ]:
# prompt: GENERATE RNN LAYERS OF 10 LAYERS

import pandas as pd
from tensorflow.keras import models, layers,preprocessing as kprocessing
import gensim.downloader
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import SimpleRNN,Dense

# ... (Your existing code) ...

input = Input(shape=(100,))
x=layers.Embedding(input_dim=embeddings.shape[0],output_dim=embeddings.shape[1],weights=[embeddings],trainable=False)(input)

# Create 10 RNN layers
for i in range(10):
    x = SimpleRNN(50, return_sequences=True, activation='tanh')(x)  # return_sequences=True for all except the last layer

# Last RNN layer without return_sequences
x = SimpleRNN(50, return_sequences=False, activation='tanh')(x)

output=Dense(5,activation='softmax')(x)

In [ ]:
model=models.Model(input,output)


In [ ]:
model.compile(loss='CategoricalCrossentropy',
optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train_vec,
batch_size=100,epochs=30,validation_split=0.1)

Epoch 1/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4221 - loss: 1.3207 - val_accuracy: 0.3780 - val_loss: 1.3223
Epoch 2/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.4424 - loss: 1.2790 - val_accuracy: 0.2028 - val_loss: 2.0278
Epoch 3/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.2438 - loss: 1.7453 - val_accuracy: 0.2795 - val_loss: 1.5457
Epoch 4/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.3585 - loss: 1.4450 - val_accuracy: 0.3819 - val_loss: 1.4241
Epoch 5/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.3834 - loss: 1.4026 - val_accuracy: 0.3740 - val_loss: 1.4109
Epoch 6/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.4568 - loss: 1.2978 - val_accuracy: 0.3602 - val_loss: 1.3916
Epoch 7/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.4392 - loss: 1.2877 - val_accuracy: 0.4016 - val_loss: 1.3074
Epoch 8/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.4621 - loss: 1.2448 - val_accuracy: 0.4134 - v

In [ ]:
scores = model.evaluate(X_test, y_test_vec,
verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Loss:%.4f%%"% scores[0])

Accuracy: 39.53%
Loss:1.3372%


In [ ]:
res=model.predict(X_test)

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [ ]:
import numpy as np
test_predictions = np.argmax(res,axis=1)


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,test_predictions)
print(cm)

[[305  82   6   5   0]
 [226  55   4   3   2]
 [ 66  12  72  12  16]
 [ 98  12  46  27  18]
 [ 88   5  54  15  41]]


In [ ]:
from sklearn.metrics import classification_report
# Print the classification report
print(classification_report(y_test,
test_predictions))

              precision    recall  f1-score   support

           0       0.39      0.77      0.52       398
           1       0.33      0.19      0.24       290
           2       0.40      0.40      0.40       178
           3       0.44      0.13      0.21       201
           4       0.53      0.20      0.29       203

    accuracy                           0.39      1270
   macro avg       0.42      0.34      0.33      1270
weighted avg       0.41      0.39      0.35      1270

